## Exercise A-Priori Algorithm

Consider the following programming exercise. Given the information of the frequent singletons $(L_1)$ and frequent pairs $(L_2)$ we compute with our previous implementation of A-Priori for k=2, implement in spark functions to compute the  **confidence** and **interest** of all the binary rules we can build from the set $L_2$. As the dataset to test your code, use the one you can find in:

https://www.kaggle.com/shazadudwadia/supermarket?select=GroceryStoreDataSet.csv

Your algorithm should follow these steps:

1. Map, using mapPartitions, each frequent pair in the RDD with L2 to its list of binary association rules (two association rules per each different frequent pair). Use then the flattened version of this RDD.
2. Map each association rule of the previous resulting RDD, to a triple with (rule,confidence,interest). Observe that you will need to use the information in L1 and the number of transactions to compute these values. You can use the version of L1 stored as a python list in the driver (so it can be passed inside functions passed to spark tasks).
3. Finally, sort the association rules by their interest, and show back in the driver program the first 10 most interesting rules

Try your algorithm with at least these three values for the parameter theta: {  0.01, 0.1, 0.2 }


Present your final notebook with all the needed code (the already provided A-priori algorithm plus your code for the association rules as requested) together with the results obtained with the different values of theta.

## High-level pseudo-code of the A-Priori algorithm

>$L_1$ := Find frequent elements (T,$\theta$)  
>k=2  
>While ($L_{k-1}$ is not empty) do:  
>>$C_k = \{ P \ | \ |P|=k, \forall S_j \subseteq P, |S_j|=k\!-\!1 \rightarrow S_j \in L_{k-1}\}$  
>>$L_k = \{ P \ | \ P \in C_k, support(P,T) \geq \theta \}$  
>>k=k+1


In [1]:
# Imports
import pyspark
import os
import math
import random
import sys

# Variables and constants
K=2
theta=0.2


In [2]:
# Init Spark Context
sc = pyspark.SparkContext('local[*]','A-Priori Algorithm')
sc

<SparkContext master=local[*] appName=A-Priori Algorithm>

**Phase 1:** Compute $L_1$ and $T_{L_1}$

In [3]:
# Read CSV and parse
transactions = sc.textFile("GroceryStoreDataSet.csv").map(lambda line: line.strip().replace("\"",'').split(","))
number_transactions = transactions.count()
print("Nº Transactions: %s\n\n%s" % (number_transactions,transactions.collect()))


Nº Transactions: 20

[['MILK', 'BREAD', 'BISCUIT'], ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'], ['BREAD', 'TEA', 'BOURNVITA'], ['JAM', 'MAGGI', 'BREAD', 'MILK'], ['MAGGI', 'TEA', 'BISCUIT'], ['BREAD', 'TEA', 'BOURNVITA'], ['MAGGI', 'TEA', 'CORNFLAKES'], ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'], ['JAM', 'MAGGI', 'BREAD', 'TEA'], ['BREAD', 'MILK'], ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'], ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'], ['COFFEE', 'SUGER', 'BOURNVITA'], ['BREAD', 'COFFEE', 'COCK'], ['BREAD', 'SUGER', 'BISCUIT'], ['COFFEE', 'SUGER', 'CORNFLAKES'], ['BREAD', 'SUGER', 'BOURNVITA'], ['BREAD', 'COFFEE', 'SUGER'], ['BREAD', 'COFFEE', 'SUGER'], ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]


In [4]:
# Compute the rdd with frequent singleton sets (L_1)
def computeL1 ( rddT, numtrans, theta ):
  rddtemp = rddT.flatMap( lambda t : [ (it,1) for it in t ] ).reduceByKey( lambda a,b : a+b  )
  return rddtemp.filter( lambda x : (float(x[1])/numtrans) >= theta )

aux_L1 = computeL1(transactions,number_transactions,theta)
print(aux_L1.collect())

[('MILK', 5), ('BREAD', 13), ('BISCUIT', 7), ('CORNFLAKES', 6), ('TEA', 7), ('MAGGI', 5), ('COFFEE', 8), ('SUGER', 6), ('BOURNVITA', 4)]


In [5]:
L1 = aux_L1.keys().collect()
print("L1 Items: %s"% L1)

L1 Items: ['MILK', 'BREAD', 'BISCUIT', 'CORNFLAKES', 'TEA', 'MAGGI', 'COFFEE', 'SUGER', 'BOURNVITA']


In [6]:
# Map any transaction to its version without elements not in L1
# L1 must be a python list, not a RDD
def computeTfilteredByL1( seqOfT, L1 ):
    for t in seqOfT:
       yield [ it for it in t if (it in L1) ]
    
TL1 = transactions.mapPartitions( lambda seqOfT : computeTfilteredByL1( seqOfT, L1 ))
print("Transactions with only frequent elements %s" % TL1.collect())

Transactions with only frequent elements [['MILK', 'BREAD', 'BISCUIT'], ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'], ['BREAD', 'TEA', 'BOURNVITA'], ['MAGGI', 'BREAD', 'MILK'], ['MAGGI', 'TEA', 'BISCUIT'], ['BREAD', 'TEA', 'BOURNVITA'], ['MAGGI', 'TEA', 'CORNFLAKES'], ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'], ['MAGGI', 'BREAD', 'TEA'], ['BREAD', 'MILK'], ['COFFEE', 'BISCUIT', 'CORNFLAKES'], ['COFFEE', 'BISCUIT', 'CORNFLAKES'], ['COFFEE', 'SUGER', 'BOURNVITA'], ['BREAD', 'COFFEE'], ['BREAD', 'SUGER', 'BISCUIT'], ['COFFEE', 'SUGER', 'CORNFLAKES'], ['BREAD', 'SUGER', 'BOURNVITA'], ['BREAD', 'COFFEE', 'SUGER'], ['BREAD', 'COFFEE', 'SUGER'], ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]


 **Phase 2:** Compute $C_2(T)$ from $T_{L_1}$

In [7]:
# For each t in seqofFilteredT (they come from T_{L_1}), compute pairs (a,b) from t that belong to C_2
def generateC2( seqofFilteredT ):
    for t in seqofFilteredT:
      cpairslist = []
      for (a,b) in [ (a,b) for i,a in enumerate(t[:-1]) for b in t[i+1:] ]:
                cpairslist.append( ((a,b),1) if (a <= b) else ((b,a),1)  )         
      yield cpairslist
    
rddC2T = TL1.mapPartitions( lambda seqOfFilteredT : generateC2( seqOfFilteredT ) )
rddC2TFlat = rddC2T.flatMap( lambda x : x )

In [8]:
print( "flattened C2T: ", rddC2TFlat.collect(), "Count: ",rddC2TFlat.count())

flattened C2T:  [(('BREAD', 'MILK'), 1), (('BISCUIT', 'MILK'), 1), (('BISCUIT', 'BREAD'), 1), (('BREAD', 'MILK'), 1), (('BISCUIT', 'BREAD'), 1), (('BREAD', 'CORNFLAKES'), 1), (('BISCUIT', 'MILK'), 1), (('CORNFLAKES', 'MILK'), 1), (('BISCUIT', 'CORNFLAKES'), 1), (('BREAD', 'TEA'), 1), (('BOURNVITA', 'BREAD'), 1), (('BOURNVITA', 'TEA'), 1), (('BREAD', 'MAGGI'), 1), (('MAGGI', 'MILK'), 1), (('BREAD', 'MILK'), 1), (('MAGGI', 'TEA'), 1), (('BISCUIT', 'MAGGI'), 1), (('BISCUIT', 'TEA'), 1), (('BREAD', 'TEA'), 1), (('BOURNVITA', 'BREAD'), 1), (('BOURNVITA', 'TEA'), 1), (('MAGGI', 'TEA'), 1), (('CORNFLAKES', 'MAGGI'), 1), (('CORNFLAKES', 'TEA'), 1), (('BREAD', 'MAGGI'), 1), (('MAGGI', 'TEA'), 1), (('BISCUIT', 'MAGGI'), 1), (('BREAD', 'TEA'), 1), (('BISCUIT', 'BREAD'), 1), (('BISCUIT', 'TEA'), 1), (('BREAD', 'MAGGI'), 1), (('MAGGI', 'TEA'), 1), (('BREAD', 'TEA'), 1), (('BREAD', 'MILK'), 1), (('BISCUIT', 'COFFEE'), 1), (('COFFEE', 'CORNFLAKES'), 1), (('BISCUIT', 'CORNFLAKES'), 1), (('BISCUIT', 'C

**Phase 3:** Compute 𝐿2 from 𝐶2(𝑇)

In [9]:
def computeL2( rddC2T, numtrans, theta ):
    pairsCountedrdd = rddC2T.reduceByKey( lambda v1,v2 : v1+v2 )
    # print(pairsCountedrdd.collect())
    # Finally, filter out from the previous rdd those pairs with frequency below theta
    return pairsCountedrdd.filter( lambda x : (float(x[1])/numtrans) >= theta )

rddL2 = computeL2( rddC2TFlat, number_transactions, theta )

In [10]:
rddL2 = rddL2.sortBy(lambda a: -a[1])
for it in rddL2.toLocalIterator():
    print (it)

(('BREAD', 'MILK'), 4)
(('BISCUIT', 'BREAD'), 4)
(('BREAD', 'TEA'), 4)
(('MAGGI', 'TEA'), 4)
(('COFFEE', 'CORNFLAKES'), 4)
(('COFFEE', 'SUGER'), 4)
(('BREAD', 'SUGER'), 4)


### Confidence

$$ \frac{support( {diapers,beer} )}{support( {diapers})} $$
where support(set) is the number of transactions where the set is found.

### Interest

$$ interest = confidence ( {diapers} \rightarrow {beer} ) - frequency( {beer} ) $$

In [11]:
def confidence(partitionData,L1):
    for element in partitionData:                                           
        yield (element[0], round(element[1] / L1[element[0][0]],2))
        
def interest(confidence, freq):
    return round(confidence - freq,2)

In [12]:
L1_dict = dict((key, value) for key, value in aux_L1.collect())

inverseL2 = rddL2.map(lambda i: ((i[0][1],i[0][0]),i[1]))

all_rules = rddL2.union(inverseL2)

confidenceRDD = all_rules.mapPartitions(lambda i: confidence(i,L1_dict))
print(confidenceRDD.collect())

[(('BREAD', 'MILK'), 0.31), (('BISCUIT', 'BREAD'), 0.57), (('BREAD', 'TEA'), 0.31), (('MAGGI', 'TEA'), 0.8), (('COFFEE', 'CORNFLAKES'), 0.5), (('COFFEE', 'SUGER'), 0.5), (('BREAD', 'SUGER'), 0.31), (('MILK', 'BREAD'), 0.8), (('BREAD', 'BISCUIT'), 0.31), (('TEA', 'BREAD'), 0.57), (('TEA', 'MAGGI'), 0.57), (('CORNFLAKES', 'COFFEE'), 0.67), (('SUGER', 'COFFEE'), 0.67), (('SUGER', 'BREAD'), 0.67)]


In [13]:
def rules(partitionData, L1, ntransaction):
    for confidence in partitionData:
        rule = str(confidence[0]).replace(',','->').replace('\'','')[1:-1]
        _confidence = confidence[1]
        _interest = interest(confidence[1], L1[confidence[0][1]]/ntransaction)
        yield (rule,_confidence,_interest)

interestRDD = confidenceRDD.mapPartitions(lambda partitionData: rules(partitionData,L1_dict,number_transactions))

In [14]:
interestRDD.sortBy(lambda a: -a[2]).take(10)

[('MAGGI-> TEA', 0.8, 0.45),
 ('TEA-> MAGGI', 0.57, 0.32),
 ('CORNFLAKES-> COFFEE', 0.67, 0.27),
 ('SUGER-> COFFEE', 0.67, 0.27),
 ('COFFEE-> CORNFLAKES', 0.5, 0.2),
 ('COFFEE-> SUGER', 0.5, 0.2),
 ('MILK-> BREAD', 0.8, 0.15),
 ('BREAD-> MILK', 0.31, 0.06),
 ('SUGER-> BREAD', 0.67, 0.02),
 ('BREAD-> SUGER', 0.31, 0.01)]

## Results obtained with the different values of theta

In [15]:
def main_method(theta):
    transactions = sc.textFile("GroceryStoreDataSet.csv").map(lambda line: line.strip().replace("\"",'').split(","))
    number_transactions = transactions.count()
    
    aux_L1 = computeL1(transactions,number_transactions,theta)
    L1_dict = dict((key, value) for key, value in aux_L1.collect())
    L1 = aux_L1.keys().collect()
    TL1 = transactions.mapPartitions( lambda seqOfT : computeTfilteredByL1( seqOfT, L1 ))
    
    rddC2T = TL1.mapPartitions( lambda seqOfFilteredT : generateC2( seqOfFilteredT ) )
    rddC2TFlat = rddC2T.flatMap( lambda x : x )
    rddL2 = computeL2( rddC2TFlat, number_transactions, theta )
    rddL2 = rddL2.sortBy(lambda a: -a[1])
    
    inverseL2 = rddL2.map(lambda i: ((i[0][1],i[0][0]),i[1]))
    all_rules = rddL2.union(inverseL2)
    
    confidenceRDD = all_rules.mapPartitions(lambda i: confidence(i,L1_dict))
    interestRDD = confidenceRDD.mapPartitions(lambda partitionData: rules(partitionData,L1_dict,number_transactions))
    print("Theta: %s - Nº Rules: %s" % (theta,interestRDD.count()))
    return interestRDD.sortBy(lambda a: -a[2]).take(10)
   

In [16]:
main_method(0.21)

Theta: 0.21 - Nº Rules: 0


[]

In [17]:
main_method(0.2)

Theta: 0.2 - Nº Rules: 14


[('MAGGI-> TEA', 0.8, 0.45),
 ('TEA-> MAGGI', 0.57, 0.32),
 ('CORNFLAKES-> COFFEE', 0.67, 0.27),
 ('SUGER-> COFFEE', 0.67, 0.27),
 ('COFFEE-> CORNFLAKES', 0.5, 0.2),
 ('COFFEE-> SUGER', 0.5, 0.2),
 ('MILK-> BREAD', 0.8, 0.15),
 ('BREAD-> MILK', 0.31, 0.06),
 ('SUGER-> BREAD', 0.67, 0.02),
 ('BREAD-> SUGER', 0.31, 0.01)]

In [18]:
main_method(0.1)

Theta: 0.1 - Nº Rules: 48


[('JAM-> MAGGI', 1.0, 0.75),
 ('COCK-> COFFEE', 1.0, 0.6),
 ('MAGGI-> TEA', 0.8, 0.45),
 ('COCK-> CORNFLAKES', 0.67, 0.37),
 ('JAM-> BREAD', 1.0, 0.35),
 ('TEA-> MAGGI', 0.57, 0.32),
 ('COCK-> BISCUIT', 0.67, 0.32),
 ('MAGGI-> JAM', 0.4, 0.3),
 ('CORNFLAKES-> COFFEE', 0.67, 0.27),
 ('SUGER-> COFFEE', 0.67, 0.27)]

In [19]:
main_method(0.01)

Theta: 0.01 - Nº Rules: 72


[('JAM-> MAGGI', 1.0, 0.75),
 ('COCK-> COFFEE', 1.0, 0.6),
 ('MAGGI-> TEA', 0.8, 0.45),
 ('COCK-> CORNFLAKES', 0.67, 0.37),
 ('JAM-> BREAD', 1.0, 0.35),
 ('TEA-> MAGGI', 0.57, 0.32),
 ('COCK-> BISCUIT', 0.67, 0.32),
 ('MAGGI-> JAM', 0.4, 0.3),
 ('CORNFLAKES-> COFFEE', 0.67, 0.27),
 ('SUGER-> COFFEE', 0.67, 0.27)]

### Theta

*Theta*, is a well-called thereshold or filter barrier. As we can see in the results, when we use **theta > 0.2**, we do not get rules because it determines the **low-pass-filter** value of support that at least appears in the transactions.

In other words, *theta* filters by specifying how popular an itemset is, as measured by the proportion of transactions in which an itemset appears.

Then we can say that when *theta* takes a smaller number, the number of rules increases, as the threshold filter is more permessive. In this way, we are able to analyse the trades and get more rules from the transactions.

Another case come when using **theta as 0.01**, the number of rules that satisfy this condition is **72**, but when we apply **theta as 0.1**, this number decreases because the frequency rules filter is more stricted as *theta* takes higher values.

### Confidence

Regarding the confidence term, refers to how likely item *Y* is purchased when item *X* has been purchased, expressed as *(X -> Y)*

So, confidence is measured by the proportion of transactions including item X, in which item Y also appears. But, in this part, *theta* does not take any consideration because the rules have been already generated.

For example, the rule *'JAM'->'MAGGI'* has 100% of confidence. This rule has the probability of 100% meaning when JAM is purchased, for sure, Maggi will be.

### Interest

Interest helps to get a better real point of view according to confidence obtained, as confidence is a biased point of view because the dataset used.

Observe that the interest can be positive, negative, or zero:

- A positive interest indicates a positive effect, i.e. that when {I} is present {j} is more likely to be present than in general.

- A negative interest indicates a negative effect, i.e. that when {I} is present {j} is less likely to be present than in general.

- A zero interest indicates no significative effect of {I} to {j}: {I} being present does not affect the frequency of {j}
